# Metodos mixtos (Mixed methods)




Consideremos el problema eliptico
\begin{equation}
\begin{array}{rclll}
\displaystyle
-\mbox{div}(c \nabla u) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & u_D & \mbox{sobre} & \partial \Omega_{D} \\
\displaystyle c \nabla u\cdot n & = & g_N & \mbox{sobre} & \partial \Omega_{N} \\
\end{array}
\end{equation}

Reescribimos la ecuacion diferencial como un sistema de primer orden introduciendo el flujo $q$, y $a = c^{-1}$
\begin{equation}
\begin{array}{rclll}
\displaystyle a q & = & \nabla u & \mbox{en} & \Omega \\
\displaystyle
\mbox{div}( q ) & = & -f & \mbox{en} & \Omega \\
\displaystyle u & = & u_D & \mbox{sobre} & \partial \Omega_{D} \\
\displaystyle q\cdot n & = & g_N & \mbox{sobre} & \partial \Omega_{N} \\
\end{array}
\end{equation}

Ahora derivamos la formulacion variacional del sistema de primer order. Primero introducimos espacios apropiados para las funciones test
\begin{equation}
\begin{array}{rcl}
\displaystyle
L_2(\Omega) & = & \{ w : \int_{\Omega} |w|^2 dx < \infty \} \\ 
H(\mbox{div},\Omega) &=& \{ v\in [L_2(\Omega)]^{d}: \mbox{div}(w)\in L_2(\Omega)\}
\end{array}
\end{equation}

Entonces la formulacion variacional mixta (busca dos soluciones) queda: Hallar $(q,u) \in H(\mbox{div}, \Omega) \times L_2(\Omega)$ con $q\cdot n = g_{N}$ sobre $\partial \Omega_{N}$ tal que

\begin{equation}
\begin{array}{rcl}
\displaystyle
\int_{\Omega} a q \cdot v\,dx + \int_{\Omega} u\,\mbox{div}(v)\,dx & = &
\displaystyle 
\int_{\partial \Omega_{D}} u_{D}v \cdot n ds \\
\displaystyle
\int_{\Omega} \mbox{div}(q)\, w\,dx & = & 
\displaystyle
- \int_{\Omega} f v \,dx
\end{array}
\end{equation}

para $(v,w) \in H(div, \Omega)\times L_2(\Omega)$ con $ v\cdot n = 0$ sobre $\partial \Omega_{N}$.

Notar que, mientras en la formulacion variacional primal la condicion de Neumann aparecia en la formulacion variacional y la condicion de Dirichlet era impueta en el espacio de funciones, en la formulacion mixta es los opuesto.

Definamos la forma bilinear $b:H(\mbox{div}, \Omega)\times L_2(\Omega) \rightarrow \mathbb R$ y el funcional $l:H(\mbox{div}, \Omega)\times L_2(\Omega)$ 

\begin{equation}
\begin{array}{rcl}
\displaystyle b((q,u), (v,w)) &:=& 
\displaystyle \int_{\Omega} a q \cdot v\,dx + \int_{\Omega} u\,\mbox{div}(v)\,dx +\int_{\Omega} \mbox{div}(q)\, w\,dx \\
\displaystyle l(v,w) &:=&
\displaystyle \int_{\partial \Omega_{D}} u_{D}v \cdot n ds - \int_{\Omega} f v \,dx
\end{array}
\end{equation}

Observemos que la forma bilineal $b$ no es coerciva, por lo tanto su analisis y la eleccion de los espacios de eleemtnos finitos no tan simple como en el caso de la formulacion primal y el metodo de elementos finitos standard.


# Elementos finitos Mixtos

Ahora definimos el metodos de elemntos finitos para subespacios $V_h\subset H(\mbox{div}, \Omega)$ y $W_h\subset L_2(\Omega)$. Hallar $(q_h,u_h)\in V_h\times W_h$ con $q_h\cdot n = P(g_n)$ sobre $\partial \Omega_N$ tal que

\begin{equation}
\begin{array}{rcl}
\displaystyle
\int_{\Omega} a q_h \cdot v\,dx + \int_{\Omega} u_h\,\mbox{div}(v)\,dx & = &
\displaystyle 
\int_{\partial \Omega_{D}} u_{D}v \cdot n ds \\
\displaystyle
\int_{\Omega} \mbox{div}(q_h)\, w\,dx & = & 
\displaystyle
- \int_{\Omega} f v \,dx
\end{array}
\end{equation}
para todo $(v,w)\in V_h\times W_h$

## Ejemplo de inestabilidad

Escojamos dos subespacios de elementos fintios $ V_h\subset H(\mbox{div}, \Omega)$ y $W_h\in L_2(\Omega)$. Por ejemplo, funciones continuas y lineales a trozos para $V_h$ y funciones discontinuas y constantes a trozos pra $W_h$, es decir
\begin{equation}
\begin{array}{rcl}
V_h &=& \{v\in [C(\Omega)]^{2}: v_i|_K\in \mathcal P_1, \,\forall K \in \mathcal T_h,\, i=1,2\}\\
W_h &=& \{w\in L_2(\Omega): w|_K \in \mathcal P_0, \, \forall K\in \mathcal T_h\}
\end{array}
\end{equation}

Caclumaos la solucion aproximada para el problema con solo condicion de frontera del tipo Dirichlet  $c =1$ y con solucion exacta $u(x,y) = x(1-x)y(1-y)$.

In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
uexact = x*(1-x)*y*(1-y)
f      = 2*y*(1-y)+2*x*(1-x)
# Space
Vh = H1(mesh, order=1)
Wh = L2(mesh, order=0)
# Mixed space
fesm = FESpace([Vh,Vh,Wh])
# variables
q1, q2, u = fesm.TrialFunction()
v1, v2, w = fesm.TestFunction()
# divergence operator
div_q = grad(q1)[0]+grad(q2)[1]
div_v = grad(v1)[0]+grad(v2)[1]
# bilinear form
am = BilinearForm(fesm)
am += (q1*v1+q2*v2 + div_q*w + div_v*u)*dx
am.Assemble()
fm = LinearForm(fesm)
fm += -f*w*dx 
fm.Assemble()

gfm = GridFunction(fesm, name="gfmixed")
gfq1, gfq2, gfu = gfm.components


#fm.vec.data -= am.mat * gfm.vec
#gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * fm.vec
solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw(uexact, mesh, 'u exact')
#Draw (gfu, mesh, "u-mixed")



![title](unstablemixed.png)
![title](uexactformixedexample.png)

## Espacios de elementos finitos para $H(\mbox{div}, \Omega)$


1. Espacio local de Raviart-Thomas para $K\in \mathcal T_h$

\begin{equation}
\begin{array}{rclrcl}
\mathcal RT_p(K)& =& \mathcal P_p(K)^{d} + x \mathcal P_{p}(K),& \mbox{dim}(\mathcal RT_p(K)) &=& d \binom{p+d}{p} + \binom{p+d-1}{p}
\end{array}
\end{equation}
y espacio $\mathcal P_p(K)$ para la variable escalar.

Espacio global de Raviart-Thomas

\begin{equation}
\mathcal RT_{p}(\mathcal T_h) = \{ v\in H(\mbox{div}, \Omega): v|_{K}\in \mathcal RT_{p}(K) \, \forall K\in \mathcal T_h \}
\end{equation}

#### Propiedad: $\mbox{div}(\mathcal RT_{p}) = \mathcal P_{p}^{d}$

2. Para $d=2$ y $p\geq 1$ y triangulos, definimos el espacio local de Brezzi-Douglas-Marini para $K\in \mathcal T_h$

\begin{equation}
\begin{array}{rclrcl}
\mathcal BDM_p(K)& =& \mathcal P_p(K)^{d},& \mbox{dim}(\mathcal BDM_p(K)) &=& (p+1)(p+2)
\end{array}
\end{equation}
y espacio $\mathcal P_{p-1}(K)$ para la variable escalar.

Espacio global de Brezzi-Douglas-Marini

\begin{equation}
\mathcal BDM_{p}(\mathcal T_h) = \{ v\in H(\mbox{div}, \Omega): v|_{K}\in \mathcal BDM_{p}(K) \, \forall K\in \mathcal T_h \}
\end{equation}



In [ ]:

# Space
# Raviart-Thomas space
Vh = HDiv(mesh,order=0, RT=True)
# Brezzi-Douglas-Marini space
#h = HDiv(mesh,order=0)
Wh = L2(mesh, order=0)
# Mixed space
fesm = FESpace([Vh,Wh])
# 
q, u = fesm.TrialFunction()
v, w = fesm.TestFunction()

# bilinear form
am = BilinearForm(fesm)
am += (q*v + div(q)*w + div(v)*u)*dx
am.Assemble()
fm = LinearForm(fesm)
fm += -f*w*dx 
fm.Assemble()

gfm = GridFunction(fesm, name="gfmixed")
gfq, gfu = gfm.components

#fm.vec.data -= am.mat * gfm.vec
#gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * fm.vec
solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw(uexact, mesh, 'u-exact')
Draw (gfu, mesh, "u-mixed")
#Redraw()

![title](RT0mixed.png)
![title](RT0mixedrefine.png)
